#My Special AI Agent

## General Setup

### Mount Google Drive
In the below code block, you will mount your Google Drive to the Google colaboratory notebook.

You will be asked to go to a URL in a new browser window, sign in to your Google account, and allow Google Drive File Stream to access your Google Drive files.

Technically, you could mount the Google Drive through the files portion of the Google Colab UI, but this method is for the purpose of being explicit.

In [ ]:
# Unnecessary to run if Google Drive is already mounted (Check files in left tab)
from google.colab import drive
drive.mount('/content/drive')

### Download Necessary Libraries/Packages

To load and use your pretrained model, you will need to have the correct libraries installed. First, the script will perform a typical update and upgrade to the packages. Then, the following libraries/packages will be installed.

Below are the necessary libraries/packages:
*   git: For using git and GitHub
*   git lfs: For using large files inside the git system
*   transformers: For using transformer based models
*   torch: For using PyTorch
*   huggingface_hub: For logging into Hugging Face

In [ ]:
!sudo apt update -y
!sudo apt upgrade -y --allow-change-held-packages

In [ ]:
#!sudo apt install git
#!git lfs install

!pip install transformers datasets
#!pip install torch
!pip install huggingface_hub
!pip install bitsandbytes
!pip install accelerate
!pip install evaluate
#!wget -nv -O- https://lambdalabs.com/install-lambda-stack.sh | sh -

### Import necessary libraries

In [ ]:
import os
import torch
import datasets
from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaConfig
from transformers import AutoTokenizer, AutoModel, pipeline, Conversation
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

CACHE_DIR = "/content/drive/MyDrive/Colab_Notebooks/AI_Models/.cache/huggingface/hub/"
os.environ["HF_HOME"] = CACHE_DIR

## AI Model Setup

### Login to Hugging Face

**Logging in to Hugging Face may be unnecessary. Return later to verify.**

In [2]:
from huggingface_hub import notebook_login

ModuleNotFoundError: ignored

In [ ]:
notebook_login()

### AI Model Folder

Navigate to AI Model stored in Google Drive.

In [ ]:
%cd /content/drive/MyDrive/Colab_Notebooks/AI_Models/WizardLM-13B-Uncensored/

/content/drive/MyDrive/Colab Notebooks/AI Models/WizardLM-13B-Uncensored


Check the current directory using the following script:

In [ ]:
%pwd

'/content/drive/MyDrive/Colab Notebooks/AI Models/WizardLM-13B-Uncensored'

Make sure the output shows the same directory as the %cd command directly above it.

#### GPU Usage

Below is a command you can run to check the specs of the GPU being run in this instance.

In [ ]:
!nvidia-smi

In [ ]:
torch.cuda.is_available                                     # check for cuda

In [ ]:
torch.cuda.get_device_name(0)

'Tesla T4'

In [3]:
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")

### Load the Model & Tokenizer

After installing the necessary libraries, you can load the model and tokenizer using the `AutoModel` and `AutoTokenizer` classes from the `transformers` library. These classes can automatically infer the correct model and tokenizer classes to use based on the config.json and tokenizer configuration files.

Here, '.' is used to indicate the current directory. If your model files are in a different directory, you should replace '.' with the path to that directory.

#### AutoTokenizer

A tokenizer is responsible for preprocessing text into an array of numbers as inputs to a model. There are multiple rules that govern the tokenization process, including how to split a word and at what level words should be split.

The most important thing to remember is you need to instantiate a tokenizer with the same model name to ensure you’re using the same tokenization rules a model was pretrained with.

In [ ]:
#import torch
#from transformers import LlamaForCausalLM, LlamaTokenizer, LlamaConfig

# To load a model from memory
#PATH = '/content/drive/MyDrive/Colab_Notebooks/AI_Models/WizardLM-13B-Uncensored/'
#model = LlamaForCausalLM.from_pretrained(PATH)
#tokenizer = LlamaTokenizer.from_pretrained(PATH)

#model = AutoModel.from_pretrained(PATH)
#model.to(device)
#tokenizer = AutoTokenizer.from_pretrained(PATH)

# To load a model from Hugging Face
model_name = "databricks/dolly-v2-3b"
question_answerer = pipeline("question-answering", model=model_name, trust_remote_code=True, device_map=0, model_kwargs={'load_in_8bit': True})

## Train the Model


Once the training is completed, we can evaluate our model and get its perplexity on the validation set like this:

In [4]:
import math
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

NameError: ignored

##Use the Model

Now that the model and tokenizer are loaded, you can use them to generate predictions. The exact way to do this will depend on what kind of model you have, but here's a simple example of how you might generate predictions from a text input:

### Provide the Prompt

In the code snippet below, you will be providing input for the model in the `prompt` variable.

In [ ]:
# Encode a text inputs
question = "What do you know?"
context = "I am a curious first-time tester."
question_answerer(question=question, context=context)

The tokenizer returns a dictionary containing:
*   input_ids: numerical representations of your tokens
*   attention_mask: indicates which tokens should be attended to

A tokenizer can also accept a list of inputs, and pad and truncate the text to return a batch with uniform length

In [ ]:
outputs = model(**prompt)
print(outputs)

### Generate Output

In the code snippet below, you will be executing the commands for the model to generate output based on the input provided previously.

In [ ]:
# Generate output from the model
#output = model(input_ids)
generate_ids = model.generate(inputs.input_ids, max_length=30)
tokenizer.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

# The output will be a tuple, with the model's forward pass output as the first item
#predictions = output[0]

## Save the Model

Once your model is fine-tuned, you can save it with its tokenizer using PreTrainedModel.save_pretrained():

In [ ]:
pt_save_directory = "./pt_save_pretrained"
tokenizer.save_pretrained(pt_save_directory)
pt_model.save_pretrained(pt_save_directory)

# Miscellaneous

Below is for training the model with datasets.

In [ ]:
from datasets import load_dataset
dataset = load_dataset("json", data_files="my_file.json")